# TP1: Algoritmos de búsqueda en Torre de Hanoi

## Integrantes

- Nicolás Rodriguez da Cruz
- Francisco Cofré
- Gaspar Acevedo Zain
- Juan Chunga
- Rodrigo Nicolás Lauro

### 1. ¿Cuáles son los PEAS de este problema? (Performance, Environment, Actuators, Sensors)

- **Performance**
  - Para el problema de Torre de Hanoi, la medida de performance son la cantidad de movimientos mínimos necesarios para transicionar entre el estado de inicial (discos ubicados en la primer varilla) al estado objetivo (todos los discos ubicados en la tercer varilla).
- **Environment**
  - Consiste en un total de tres varillas y cinco discos. Inicialmente, los cinco discos están ubicados en la primer varilla (estado inicial).
- **Actuators**
  - Son el conjunto de libraries (estructura de datos, funciones, clases, etc.) que permiten realizar los pasos necesarios para transicionar entre estados del problema.
- **Sensors**
  - Son el conjunto de libraries (estructura de datos, funciones, clases, etc.) que permiten leer el estado actual del problema, es decir, la ubicación de los cinco discos en las tres varillas en un momento dado.

### 2. ¿Cuáles son las propiedades del entorno de trabajo?
- **Totalmente observable**: ya que los sensores del agente le permiten conocer el estado de manera completa en cualquier momento, es decir, como se distribuyen los cinco discos entre las tres varillas.
- **Determinista**: ya que ante un estado *origen* determinado, usando un mismo algoritmo determinado, siempre se ejecutará la misma acción para transicionar a un mismo estado *destino*.
- **Secuencial**: ya que cada movimiento que realiza el agente afecta a los estados posteriores.
- **Estático**: ya que el ambiente/entorno no cambia sin acciones del agente.
- **Discreto**: ya que los estados del problema y acciones que realiza el agente para llegar a ellos son finitos.
- **Agente individual**: ya que hay un solo agente que interactúa con el ambiente/entorno.

### 3. En el contexto de este problema, defina los siguientes conceptos:
- **Estado**: corresponde a la ubicación de cada disco en función de las varillas, en un momento dado.
- **Espacio de estados**: corresponde a las distintas combinaciones de estados.
- **Árbol de búsqueda**: estructura en la que cada nodo representa un estado y las aristas, acciones.  
- **Nodo de búsqueda**: elemento del árbol que contiene estado, padre, acción y costo acumulado.  
- **Objetivo**: consiste en el estado final que soluciona al problema de Torres de Hanoi, siendo este el que contiene todos los discos en la última varilla.
- **Acción**: consiste en las operaciones que realiza el agente para pasar de un estado a otro, respetando la regla de no posicionar un disco más grande sobre otro más pequeño.
- **Frontera**: consiste en los nodos que genera el algoritmo para explorar, pero que no fueron expandidos.

### 4. Implementación del algoritmo: IDDFS

Se utiliza **Iterative Deepening Depth-First Search (IDDFS)** para encontrar la solución óptima. `IDDFS` funciona de forma tal que ejecuta sucesivas búsquedas en profundidad con un límite creciente de profundidad (0, luego 1, luego 2, …) hasta encontrar el estado objetivo. En cada iteración, realiza una `DFS` “acotada” que explora tan profundo como lo permita el límite actual y luego retrocede, combinando la baja memoria de `DFS` ($O(b·d)$, donde b es el factor de ramificación y d la profundidad de la solución) con la garantía de optimalidad de BFS en espacios de estado de coste uniforme.

In [1]:
def successors(state, num_pegs=3, num_disks=5):
    """Genera estados sucesores de un estado dado."""
    succ = []
    for peg in range(num_pegs):
        disks_on_peg = [i for i, p in enumerate(state) if p == peg]
        if not disks_on_peg:
            continue
        top_disk = min(disks_on_peg)
        for dest in range(num_pegs):
            if dest == peg:
                continue
            disks_on_dest = [i for i, p in enumerate(state) if p == dest]
            if disks_on_dest and min(disks_on_dest) < top_disk:
                continue
            new_state = list(state)
            new_state[top_disk] = dest
            succ.append(tuple(new_state))
    return succ

def recursive_dls(state, goal, limit, path):
    if state == goal:
        return path
    elif limit == 0:
        return 'cutoff'
    else:
        cutoff_occurred = False
        for child in successors(state):
            if child in path:
                continue
            result = recursive_dls(child, goal, limit - 1, path + [child])
            if result == 'cutoff':
                cutoff_occurred = True
            elif result is not None:
                return result
        return 'cutoff' if cutoff_occurred else None

def iddfs(initial, goal):
    depth = 0
    while True:
        result = recursive_dls(initial, goal, depth, [initial])
        if result != 'cutoff' and result is not None:
            return result
        depth += 1

disks = 5
initial_state = tuple([0] * disks)
goal_state = tuple([2] * disks)

demo_path = iddfs(initial_state, goal_state)
print(f"Solución encontrada en {len(demo_path)-1} movimientos.")

Solución encontrada en 31 movimientos.


### 5. Complejidad teórica

- **Tiempo**
  - $O(b^d)$
    - `b` o **branching factor**: el número máximo de sucesores para un estado
    - `d` o **profundidad de solución**: es decir, la longitud del camino más corto del árbol de búsqueda
  - Cada vez que sube un nivel de profundidad (de 0 a 1, de 1 a 2, … hasta la solución), vuelve a explorar todos los caminos anteriores. Cuanto más lejos esté la solución, más crece el trabajo: agregar un disco extra puede llegar a duplicar aproximadamente el esfuerzo. Es decir, el tiempo necesario aumenta muy rápido (de forma exponencial) al sumar discos.

- **Memoria**
  - $O(b*d)$
    - `b` o **branching factor**: el número máximo de sucesores para un estado
    - `d` o **profundidad de solución**: es decir, la longitud del camino más corto del árbol de búsqueda
  - En lugar de recordar todos los estados de un mismo nivel (como hace BFS), IDDFS solo necesita guardar la “ruta” actual desde el inicio hasta donde está profundizando en ese momento. Si la solución final son 31 movimientos, en memoria habrá como máximo 31 estados apilados. Así, el uso de memoria crece de manera moderada, directamente relacionado con cuántos movimientos (o discos) haya, no con todos los posibles estados juntos.

### 6. Medición de tiempo y memoria del algoritmo

**Consumo de Memoria**
En un total de diez ejecuciones, el algoritmo `IDDFS` consume, en KiB:
- En **promedio**, `12.98 KiB`, con un **Desvío estandar** de `1.56 KiB`

**Tiempo de ejecución**
En un total de diez ejecuciones, el algoritmo `IDDFS` tardó, en segundos:
- En **promedio** `1931.5016 segundos`, con un **Desvío estandar** de `46.55 segundos`

In [2]:
import time
import tracemalloc
import statistics

times = []
memories = []
for _ in range(10):
    tracemalloc.start()
    t0 = time.perf_counter()
    path = iddfs(initial_state, goal_state)
    t1 = time.perf_counter()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    times.append(t1 - t0)
    memories.append(peak)

avg_time = statistics.mean(times)
stdev_time = statistics.stdev(times)
avg_mem = statistics.mean(memories)
stdev_mem = statistics.stdev(memories)

print(f"Tiempo promedio: {avg_time:.6f}s ± {stdev_time:.6f}s")
print(f"Memoria pico promedio: {avg_mem/1024:.2f} KiB ± {stdev_mem/1024:.2f} KiB")

Tiempo promedio: 1931.501635s ± 46.550089s
Memoria pico promedio: 12.98 KiB ± 1.56 KiB


### 7. Comparación con la solución óptima

Siendo la solución óptima $2^k – 1$, empleando ***5*** discos como ***k*** esta queda en ***31 movimientos***.

Para el problema de Torres de Hanoi con las siguientes características:
- Cantidad de discos: cinco (5)
- Cantidad de varillas: tres (3)
- Algoritmo: `IDDFS`

se logró alcanzar la solución en la cantidad de movimientos óptimos: ***31 movimientos***.

In [ ]:
import statistics

disks = 5
initial_state = tuple([0] * disks)
goal_state = tuple([2] * disks)

results = []
results_len = []
for _ in range(10):
    path = iddfs(initial_state, goal_state)

    results.append(path)
    results_len.append(len(path) -1)

avg_res_len = statistics.mean(results_len)
stdev_res_len = statistics.stdev(results_len)

print(f"Cantidad de movimientos promedio en 10 ejecuciones: {avg_res_len} ± {stdev_res_len}")
